In [1]:
%config IPCompleter.greedy=True

In [2]:
import sys, os, re
import locale
import math

In [3]:
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

# input_data.py contains our sample data, including hull quantites and me values

In [4]:
import input_data

### what mineral types do we need to build these hulls (and what are their prices)

In [5]:
mineral_ids = sorted(list(set((sum([[m for m in input_data.ore_yield_dict[o].keys()] for o in input_data.ore_yield_dict.keys()], [])))))
mineral_names = [input_data.item_info_dict[m].get('name', '') for m in mineral_ids]
mineral_prices = [input_data.item_prices_dict[m] for m in mineral_ids]

In [6]:
dict(zip(mineral_names, mineral_prices))

{'Tritanium': 6.32,
 'Pyerite': 17.5,
 'Mexallon': 137.7,
 'Isogen': 36.5,
 'Nocxium': 1159.0,
 'Zydrine': 1335.0,
 'Megacyte': 627.8}

### how many of those types to build what we want

In [7]:
def calc_qty(count, quantity, me, fm):
    return round(max(float(count),
        math.ceil(round(float(count)*(float(quantity)*(1.0-float(me)/100.0)*float(fm)), 2))))


# add up the minerals for each ship order
total_minerals = dict(zip(mineral_ids, [0] * len(mineral_ids)))
for ship_id in input_data.ship_build_dict.keys():
    ship_qty = input_data.ship_build_dict.get(ship_id, 0)
    ship_me = input_data.ship_me_dict.get(ship_id, 0)
    ship_minerals = dict(zip(mineral_ids,
        [calc_qty(ship_qty, input_data.ship_requirements_dict[ship_id].get(m, 0), ship_me, 1.0) for m in sorted(list(mineral_ids))])
        )
    print("{:12s} x {:3} ({:2}) = {}".format(input_data.item_info_dict[ship_id]['name'], ship_qty, ship_me, ship_minerals))
    total_minerals = dict(zip(mineral_ids, 
        [total_minerals[x] + ship_minerals[x] for x in mineral_ids]
    ))

print("{:12s}            = {}".format('Total', total_minerals))

required_minerals = [total_minerals.get(m, 0) for m in mineral_ids]

Thrasher     x  10 (10) = {34: 431163, 35: 103347, 36: 35793, 37: 15813, 38: 10, 39: 1620, 40: 234}
Vexor        x   1 (10) = {34: 560000, 35: 120000, 36: 37000, 37: 9100, 38: 2601, 39: 1181, 40: 321}
Total                   = {34: 991163, 35: 223347, 36: 72793, 37: 24913, 38: 2611, 39: 2801, 40: 555}


In [8]:
print(required_minerals)

[991163, 223347, 72793, 24913, 2611, 2801, 555]


### how much would it cost to just buy the minerals directly

In [9]:
minerals_cost = int(sum([required_minerals[mi] * mineral_prices[mi] for mi in range(len(mineral_ids))]))

In [10]:
print("minerals_cost: {:n} ISK".format(minerals_cost))

minerals_cost: 28,219,556 ISK


### refining efficiency - this is the variable that makes the most difference

In [11]:
refining_efficiency = 0.50

### inputs (and their yield) - some experimentation here to reduce the number of inputs to make the optimization problem smaller (faster)

In [12]:
ore_ids = sorted(list(filter(lambda x: input_data.item_info_dict[x].get('name', '').split()[0] == 'Compressed', list(input_data.ore_yield_dict.keys()))))

# cut back the list of ore so we only have the 'basic' compressed types. still works if you skip this next line, but larger data-set -> harder to verify manually.
ore_ids = sorted(list(filter(lambda x: len(input_data.item_info_dict[x].get('name', '').split()) == 2, ore_ids)))

ore_yields = dict(zip(ore_ids, [dict(zip(mineral_ids, [math.floor(refining_efficiency * input_data.ore_yield_dict[o].get(m, 0)) for m in mineral_ids])) for o in ore_ids]))
ore_names = [input_data.item_info_dict[o].get('name', '') for o in ore_ids]
ore_packaged_volume = [input_data.item_info_dict[o].get('packagedVolume', '') for o in ore_ids]
ore_prices = [input_data.item_prices_dict[o] for o in ore_ids]

In [13]:
dict(zip(ore_names, ore_prices))

{'Compressed Arkonor': 368500.0,
 'Compressed Bistot': 331800.0,
 'Compressed Crokite': 953400.0,
 'Compressed Gneiss': 330700.0,
 'Compressed Hedbergite': 249900.0,
 'Compressed Hemorphite': 198900.0,
 'Compressed Jaspet': 109400.0,
 'Compressed Kernite': 26500.0,
 'Compressed Omber': 15180.0,
 'Compressed Spodumain': 1217000.0,
 'Compressed Plagioclase': 7890.0,
 'Compressed Pyroxeres': 5345.0,
 'Compressed Scordite': 2367.0,
 'Compressed Veldspar': 1902.0}

In [14]:
ore_yields

{28367: {34: 0, 35: 1600, 36: 600, 37: 0, 38: 0, 39: 0, 40: 60},
 28388: {34: 0, 35: 1600, 36: 600, 37: 0, 38: 0, 39: 80, 40: 0},
 28391: {34: 0, 35: 400, 36: 1000, 37: 0, 38: 400, 39: 0, 40: 0},
 28397: {34: 0, 35: 1000, 36: 750, 37: 400, 38: 0, 39: 0, 40: 0},
 28401: {34: 0, 35: 225, 36: 0, 37: 0, 38: 60, 39: 0, 40: 0},
 28403: {34: 0, 35: 0, 36: 0, 37: 120, 38: 45, 39: 0, 40: 0},
 28406: {34: 0, 35: 0, 36: 75, 37: 0, 38: 25, 39: 0, 40: 0},
 28410: {34: 0, 35: 0, 36: 30, 37: 60, 38: 0, 39: 0, 40: 0},
 28416: {34: 0, 35: 45, 36: 0, 37: 37, 38: 0, 39: 0, 40: 0},
 28420: {34: 24000, 35: 0, 36: 0, 37: 500, 38: 80, 39: 40, 40: 20},
 28422: {34: 87, 35: 0, 36: 35, 37: 0, 38: 0, 39: 0, 40: 0},
 28424: {34: 0, 35: 45, 36: 15, 37: 0, 38: 0, 39: 0, 40: 0},
 28429: {34: 75, 35: 45, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
 28432: {34: 200, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0}}

In [15]:
import pymprog

In [16]:
def calc_optimal_ores(mineral_ids, ore_ids, required_minerals, ore_prices_list, ore_yield_list):
    p = pymprog.model('MineralExplOrer')
    p.verbose(False)
    x = p.var('x', len(ore_ids), kind=int)
    p.minimize(sum(x[oi] * ore_prices_list[oi] for oi in range(len(ore_ids))))
    [[sum(x[oi]*ore_yield_list[oi][mi] for oi in range(len(ore_ids))) >= required_minerals[mi]] for mi in range(len(mineral_ids))]
    p.solver(solver='simplex', msg_lev=pymprog.glpk.GLP_MSG_OFF)
    p.solve()
    results = [0.0 for oi in range(len(ore_ids))]
    if p.get_status() in [ pymprog.glpk.GLP_OPT , pymprog.glpk.GLP_FEAS ]:
        results = [int(x[oi].primal) for oi in range(len(ore_ids))]
    p.end()
    return results

In [17]:
print("mineral_ids: {}".format(mineral_ids))

mineral_ids: [34, 35, 36, 37, 38, 39, 40]


In [18]:
print("ore_ids: {}".format(ore_ids))

ore_ids: [28367, 28388, 28391, 28397, 28401, 28403, 28406, 28410, 28416, 28420, 28422, 28424, 28429, 28432]


In [19]:
print("required_minerals: {}".format(required_minerals))

required_minerals: [991163, 223347, 72793, 24913, 2611, 2801, 555]


In [20]:
print("ore_prices: {}".format(ore_prices))

ore_prices: [368500.0, 331800.0, 953400.0, 330700.0, 249900.0, 198900.0, 109400.0, 26500.0, 15180.0, 1217000.0, 7890.0, 5345.0, 2367.0, 1902.0]


In [21]:
ore_yield_list = [[ore_yields[o][i] for i in mineral_ids] for o in ore_ids]
print("ore_yield_list: {}".format(ore_yield_list))

ore_yield_list: [[0, 1600, 600, 0, 0, 0, 60], [0, 1600, 600, 0, 0, 80, 0], [0, 400, 1000, 0, 400, 0, 0], [0, 1000, 750, 400, 0, 0, 0], [0, 225, 0, 0, 60, 0, 0], [0, 0, 0, 120, 45, 0, 0], [0, 0, 75, 0, 25, 0, 0], [0, 0, 30, 60, 0, 0, 0], [0, 45, 0, 37, 0, 0, 0], [24000, 0, 0, 500, 80, 40, 20], [87, 0, 35, 0, 0, 0, 0], [0, 45, 15, 0, 0, 0, 0], [75, 45, 0, 0, 0, 0, 0], [200, 0, 0, 0, 0, 0, 0]]


In [22]:
rval = calc_optimal_ores(mineral_ids, ore_ids, required_minerals, ore_prices, ore_yield_list)

In [23]:
print(rval)

[10, 36, 7, 0, 0, 0, 0, 414, 2, 0, 736, 1, 3263, 3413]


In [24]:
actual_ore_cost = int(sum([rval[oi] * ore_prices[oi] for oi in range(len(ore_ids))]))
actual_ore_packaged_volume = int(sum([rval[oi] * ore_packaged_volume[oi] for oi in range(len(ore_ids))]))
print("actual_ore: {}".format(dict(zip(ore_names, rval))))
print("actual_ore_cost: {:n} ISK".format(actual_ore_cost))
print("actual_ore_packaged_volume: {:n} m3".format(actual_ore_packaged_volume))

actual_ore: {'Compressed Arkonor': 10, 'Compressed Bistot': 36, 'Compressed Crokite': 7, 'Compressed Gneiss': 0, 'Compressed Hedbergite': 0, 'Compressed Hemorphite': 0, 'Compressed Jaspet': 0, 'Compressed Kernite': 414, 'Compressed Omber': 2, 'Compressed Spodumain': 0, 'Compressed Plagioclase': 736, 'Compressed Pyroxeres': 1, 'Compressed Scordite': 3263, 'Compressed Veldspar': 3413}
actual_ore_cost: 53,332,392 ISK
actual_ore_packaged_volume: 1,622 m3


In [25]:
for oi in range(len(ore_ids)):
    if rval[oi] > 0:
        print("{},{}".format(ore_names[oi], rval[oi]))

Compressed Arkonor,10
Compressed Bistot,36
Compressed Crokite,7
Compressed Kernite,414
Compressed Omber,2
Compressed Plagioclase,736
Compressed Pyroxeres,1
Compressed Scordite,3263
Compressed Veldspar,3413


In [26]:
actual_minerals = [sum([rval[oi] * ore_yields[ore_ids[oi]][m] for oi in range(len(ore_ids))]) for m in mineral_ids]
actual_minerals_cost = int(sum([actual_minerals[mi] * mineral_prices[mi] for mi in range(len(mineral_ids))]))
print("actual_minerals: {}".format(dict(zip(mineral_names, actual_minerals))))
print("actual_minerals_cost: {:n} ISK".format(actual_minerals_cost))

actual_minerals: {'Tritanium': 991357, 'Pyerite': 223370, 'Mexallon': 72795, 'Isogen': 24914, 'Nocxium': 2800, 'Zydrine': 2880, 'Megacyte': 600}
actual_minerals_cost: 28,574,263 ISK


In [27]:
residual_minerals = [actual_minerals[mi] - required_minerals[mi] for mi in range(len(mineral_ids))]
minerals_slippage = int(sum([residual_minerals[mi] * mineral_prices[mi] for mi in range(len(mineral_ids))]))
print("residual_minerals: {}".format(dict(zip(mineral_names, residual_minerals))))
print("minerals_slippage: {:n} ISK".format(minerals_slippage))

residual_minerals: {'Tritanium': 194, 'Pyerite': 23, 'Mexallon': 2, 'Isogen': 1, 'Nocxium': 189, 'Zydrine': 79, 'Megacyte': 45}
minerals_slippage: 354,707 ISK
